In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import warnings
# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Could not infer format")

In [2]:
# Step 2: Data Analysis 

# Load the combined dataset in Parquet format
combined_df = pd.read_parquet('combined_dataset.parquet')

# Filter for active households
active_households_df = combined_df[combined_df['Active HH'] == 1]

In [3]:
# 1. What is the average number of cars per household?


# Group by household ID and count the number of unique car IDs
cars_per_household = active_households_df.groupby('HH_ID')['CAR_ID'].nunique()

# Calculate the average number of cars per household
average_cars_per_household = cars_per_household.mean()

# Display the result
print(f"The average number of cars per active household is: {average_cars_per_household:.2f}")

The average number of cars per active household is: 3.81


In [4]:
# 2. How many cars are there by model year?
# Group by model year and count the number of cars
cars_by_model_year = active_households_df['Model Year'].value_counts().sort_index(ascending=False)

# Display the top 10 results
print("Number of cars by model year (sorted by year, descending):")
print(cars_by_model_year.head(10))

Number of cars by model year (sorted by year, descending):
Model Year
2024    22195
2023    21979
2022    22041
2021    22289
2020    21909
2019    21990
2018    21768
2017    22386
2016    22328
2015    22142
Name: count, dtype: int64


In [5]:
# 3. How many cars are there by make?
# Group by 'Make' and count the number of cars
cars_by_make = active_households_df['Make'].value_counts()

# Display the result
print("Number of cars by make:")
print(cars_by_make)

Number of cars by make:
Make
Manufacturer2    135862
Manufacturer1    135577
Manufacturer3    135326
Manufacturer6     88643
Manufacturer7     88553
Manufacturer4     47160
Manufacturer5     47090
Name: count, dtype: int64


In [6]:
# 4. Which cars are the safest?  What variables did you consider to define “safe”? 
# Consider only cars with both Vehicle Safety Discount and Driver Safety Discount
safe_cars_df = active_households_df[
    (active_households_df['Vehicle Safety Discount'] == 1) & 
    (active_households_df['Driver Safety Discount'] == 1)
]

# Rank cars by Claim Payout, assuming lower payouts indicate safer cars
safe_cars_df = safe_cars_df.sort_values(by='Claim Payout', ascending=True)

# Display the top safest cars
print("Top safest cars based on Vehicle Safety Discount, Driver Safety Discount, and Claim Payout:")
print(safe_cars_df[['Make', 'Model Year', 'Body Style', 'Claim Payout']].head(10))

Top safest cars based on Vehicle Safety Discount, Driver Safety Discount, and Claim Payout:
                 Make  Model Year Body Style  Claim Payout
16      Manufacturer2        2007     4 door             0
562919  Manufacturer3        1971        SUV             0
562926  Manufacturer2        1972        SUV             0
562929  Manufacturer2        2007        SUV             0
562939  Manufacturer3        2001     4 door             0
562943  Manufacturer3        2018        SUV             0
562954  Manufacturer1        2014     4 door             0
562956  Manufacturer7        2023     4 door             0
562960  Manufacturer5        2006     4 door             0
562963  Manufacturer6        2005        SUV             0


In [7]:
# 5. Which states have the largest households (defined as number of customers in a household)?
# Group by household ID (HH_ID) and count the number of unique customers (CUST_ID)
household_sizes = active_households_df.groupby('HH_ID')['CUST_ID'].nunique()

# Merge the household size back to the main DataFrame to associate with state information
household_sizes_df = active_households_df[['HH_ID', 'State_x']].drop_duplicates().merge(
    household_sizes.rename('Household Size'), on='HH_ID'
)

# Group by state and calculate the average household size
state_household_sizes = household_sizes_df.groupby('State_x')['Household Size'].mean()

# Sort by household size in descending order to see which states have the largest households
state_household_sizes = state_household_sizes.sort_values(ascending=False)

# Display the top states with the largest households
print("States with the largest households (average number of customers per household):")
print(state_household_sizes.head(10))

States with the largest households (average number of customers per household):
State_x
AR    3.876744
MA    3.862181
NE    3.855932
MD    3.855244
WA    3.853886
ME    3.852683
HI    3.852651
AL    3.849607
MI    3.849462
NV    3.848837
Name: Household Size, dtype: float64


In [8]:
# 6. How many active households are there as of 1/1/2021?
# Convert 'HH Start Date' to datetime format with .loc to avoid SettingWithCopyWarning
active_households_df.loc[:, 'HH Start Date'] = pd.to_datetime(active_households_df['HH Start Date'], errors='coerce')

# Define the cutoff date
cutoff_date = pd.to_datetime('2021-01-01')

# Filter out any rows where the date conversion failed (resulting in NaT)
active_households_df = active_households_df.dropna(subset=['HH Start Date'])

# Filter for households that were active as of 1/1/2021
active_households_as_of_2021 = active_households_df[
    (active_households_df['HH Start Date'] <= cutoff_date) & 
    (active_households_df['Active HH'] == 1)
]

# Get the number of unique active households
num_active_households = active_households_as_of_2021['HH_ID'].nunique()

# Display the result
print(f"Number of active households as of 1/1/2021: {num_active_households}")

Number of active households as of 1/1/2021: 96421


In [9]:
# 7. What is the average age of customers?
# Convert 'Date of Birth' to datetime format
active_households_df['Date of Birth'] = pd.to_datetime(active_households_df['Date of Birth'], errors='coerce')

# Calculate the age of each customer
current_year = pd.to_datetime('today').year
active_households_df['Age'] = current_year - active_households_df['Date of Birth'].dt.year

# Filter out unrealistic ages (e.g., less than 18 years)
valid_ages_df = active_households_df[active_households_df['Age'] >= 18]

# Calculate the average age, ignoring NaN values
average_age = valid_ages_df['Age'].mean()

# Display the result
print(f"The average age of customers is: {average_age:.2f} years")

The average age of customers is: 31.80 years


In [10]:
# 8. How much does age vary by region?
# Group by region (assuming 'State_x' represents region) and calculate the standard deviation of ages
age_variation_by_region = valid_ages_df.groupby('State_x')['Age'].std()

# Display the results
print("Age variation by region (standard deviation):")
print(age_variation_by_region.sort_values(ascending=False))

Age variation by region (standard deviation):
State_x
UT    9.381419
WI    9.266581
AL    9.199879
NY    9.194210
ID    9.175519
CO    9.174811
AZ    9.154639
SC    9.151280
NH    9.147248
KY    9.143139
IN    9.131519
MT    9.129278
WV    9.108490
DE    9.100462
MA    9.096284
NM    9.093254
RI    9.093189
HI    9.087988
ND    9.074230
NE    9.073035
WY    9.065078
IL    9.063765
NJ    9.063647
LA    9.061940
ME    9.059470
FL    9.058000
NC    9.056540
KS    9.054822
MO    9.049148
CA    9.048333
TX    9.041147
CT    9.040568
IA    9.040369
MI    9.038803
MD    9.037131
TN    9.035910
VA    9.033866
VT    9.032301
AK    9.026255
MS    9.024807
OR    9.020563
OK    9.019455
SD    9.009976
AR    9.009898
NV    9.006045
PA    9.004461
OH    8.993309
WA    8.977569
MN    8.975167
GA    8.965214
Name: Age, dtype: float64


In [11]:
# 9. Which age group has the most expensive claims? 
# Convert 'Date of Birth' to datetime format (if not already done)
active_households_df['Date of Birth'] = pd.to_datetime(active_households_df['Date of Birth'], errors='coerce')

# Calculate the age of each customer (if not already done)
current_year = pd.to_datetime('today').year
active_households_df['Age'] = current_year - active_households_df['Date of Birth'].dt.year

# Filter out records where age is under 18
valid_ages_df = active_households_df[active_households_df['Age'] >= 18].copy()

# Define age groups (bins), combining all ages 73 and above into one group
bins = [18, 23, 28, 33, 38, 43, 48, 53, 58, 63, 68, 73, 100]
labels = ['18-22', '23-27', '28-32', '33-37', '38-42', '43-47', '48-52', '53-57', '58-62', '63-67', '68-72', '73+']

# Use these bins and labels for categorizing ages
valid_ages_df.loc[:, 'Age Group'] = pd.cut(valid_ages_df['Age'], bins=bins, labels=labels, right=False)

# Calculate the average claim payout per age group
avg_claims_by_age_group = valid_ages_df.groupby('Age Group', observed=False)['Claim Payout'].mean()

# Remove NaN values
avg_claims_by_age_group = avg_claims_by_age_group.dropna()

# Display the results
print("Average claim payout by age group (NaN values removed):")
print(avg_claims_by_age_group.sort_values(ascending=False))

Average claim payout by age group (NaN values removed):
Age Group
43-47    1358.962141
33-37    1269.128386
23-27    1245.927420
18-22    1241.074056
28-32    1160.118583
48-52    1145.497340
38-42    1023.299820
Name: Claim Payout, dtype: float64
